In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
import datetime as dt  


driver2 = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")
#driver2 = webdriver.Chrome()
driver2.get("http://www.ncsl.org/research/energy/energy-environment-legislation-tracking-database.aspx")

#SEARCH PAGE
#click checkbox to search by Renewable Energy - Solar
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_27").click()
#click checkbox to search by Green Jobs
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_13").click()
#click checkbox to search by Energy Security and Critical Infrastructure
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_8").click()
#click checkbox to search by Financing Energy Efficiency and Renewable Energy
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_9").click()
#click checkbox to search by Utility Regulation
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_31").click()
#click checkbox to search by Electric Grid and Transmission
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_5").click()
#click checkbox to search by Energy Efficiency - Building Codes and Standards
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_7").click()
#click checkbox to search by Climate Change - Adaptation and Environment
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_1").click()
#click checkbox to search by Climate Change - Carbon Capture and Sequestration
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_2").click()
#click checkbox to search by Climate Change - Emissions Reduction
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_3").click()
#click checkbox to search by Fossil Energy - Coal
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_11").click()
#click checkbox to search by Fossil Energy – Natural Gas
driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxTopics_12").click()

#select "All States"                                               
element3 = driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_ckBxAllStates")
element3.click()
#click to submit search
element = driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_btnSearch")
element.click()

In [2]:
#Results Page

#Get number of states and bills
sum_blob = driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_resultsHeader")
sum_clean_blob = sum_blob.text.split()
num_states = sum_clean_blob[3]
num_bills = sum_clean_blob[6]

print ("Total States: " + num_states)
print ("Total Bills: " + num_bills)

Total States: 49
Total Bills: 1765


In [3]:
import csv
with open('C:/Users/KNguyen/NCSL_list.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['Bill Name', 'Bill State', 'Bill Chamber', 'Bill Number', 'Bill_One', 
                  'Bill Year', 'Title', 
                  'Status', 'Status_One', 'Status_Two', 'Status_Three',
                  'Date of last action', 
                  'Authors Name', "Author's Affiliation", 
                  'Coauthor 1', 'Affiliation',
                  'Coauthor 2', 'Affiliation',
                  'Coauthor 3', 'Affiliation',
                  'Other Coauthors',
                  'Topics', 'Category 1', 'Category 2', 'Category 3',
                  'Associated Bills', 'Summary','History', 'Updates', 'Date added to DB']]
    results_dirty = driver2.find_element_by_css_selector("#dnn_ctr71217_StateNetDB_linkList")
    results_html = results_dirty.get_attribute("innerHTML")
    results_segs = results_html.split("<hr>")
    #for loop through each bill and spit into small sections
    for seg in results_segs:
        #clean up..
        bill = seg.split("<div>")
        bill_clean = bill[1].replace("\n","").split("<br>")
        #put clean parts into a list called 'lion'
        lion = []
        for bit in bill_clean:
            format_bit = bit.split()
            lion.append(format_bit)  
        #Create columns Bill State, Chamber, and Number
        bill_state = lion[0][0]
        bill_chamber = ""
        bill_number = ""
        if (len(lion[0])==2):
            bill_number = lion[0][1]
        if (len(lion[0])==3):
            bill_chamber = lion[0][1]
            bill_number = lion[0][2]
        bill_one = bill_chamber + bill_number
        bill_name = bill_state + " " + bill_chamber + " " + bill_number
        #print("Bill Name: " + bill_name)
        #Create column Bill Year
        bill_year = lion[1][0]
        #print ("Bill Year: " + bill_year)
        #Create column Title
        title = ""
        for i in range (4,len(lion[2])):
            title += lion[2][i] + " "
        #print ("Title: " + title)
        #Create column Status and other Status subcolumns
        status = ""
        for i in range (2,len(lion[3])):
            status += lion[3][i] + " "
        complete_status = status.split('-')
        status_one = complete_status[0]
        status_two = complete_status[1]
        status_three = ""
        if (len(complete_status)>2):
            status_three = complete_status[2]
        #print ("Status: " + status)
        #Create Date of Last Action by checking the position of "-"
        #4 outcomes: date, Enacted and Vetoed w/ date, and LineItemVetoed w/ date
        if ("-" == lion[4][-2]):
            date = ''.join(lion[4][-3:(len(lion[4]))])
        elif ("-" == lion[4][-4]):
            date = ''.join(lion[4][-5:(len(lion[4]))])
        else:
            date = lion[4][len(lion[4])-1]
        #print ("Date of last action: " + date)
        #Seperate Authors and Coauthors by looking for "Additional"
        authors = ""
        coauthors = ""
        final_authors = []
        complete_coauthors = []
        #clean up authors list..
        for people in lion[5]:
            clean_people = people.replace("<b>","").replace("</b>","")
            final_authors.append(clean_people) 
        #Look for coauthors
        if "Additional" in final_authors:
            for i in range (1,final_authors.index("Additional")):
                authors += final_authors[i] + " "
            #print ("Author: " + authors)
            for i in range (final_authors.index("Additional")+2,len(final_authors)):
                coauthors += final_authors[i] + " "
            #print ("Coauthors: " + coauthors)                 
        else:
            for i in range (1,len(final_authors)):
                authors += final_authors[i] + " "
            #print ("Author: " + authors)
        #split author name and affiliation:
        authors_name = ""
        aff_authors = ""
        if ("(" in authors):
            aff_authors = authors.split('(')[1].split(")")[0]
            authors_name = authors.split(" ")[0]
        else:
            authors_name = authors
        #split 3 coauthors name and affilication:
        aff_temp = coauthors.split(";")
        aff_coauthors = []
        other_coauthors = ""
        coauthors1_name = ""
        coauthors2_name = ""
        coauthors3_name = ""
        aff_coauthors1 = ""
        aff_coauthors2 = ""
        aff_coauthors3 = ""
        if ("(" in coauthors):
            for co in aff_temp:
                aff = co.split('(')[1].split(")")[0]
                fi_co = co.split(" ")[0]
                aff_coauthors.append(aff)
                complete_coauthors.append(fi_co)
            for i in range (3,len(aff_temp)):
                other_coauthors += aff_temp[i] + " "
            coauthors1_name = complete_coauthors[0]
            aff_coauthors1 = aff_coauthors[0]
            if (len(complete_coauthors)>1):
                coauthors2_name = complete_coauthors[1]
                aff_coauthors2 = aff_coauthors[1]
            else:
                coauthors2_name = " "
                aff_coauthors2 = " "
            if (len(complete_coauthors)>2):
                coauthors3_name = complete_coauthors[2]
                aff_coauthors3 = aff_coauthors[2]
            else:
                coauthors3_name = " "
                aff_coauthors3 = " "
        else:
            aff_coauthors = []
            other_coauthors = ""
        #Create column Topics                    
        topics = ""
        for i in range (2, len(lion[6])):
            topics += lion[6][i] + " "
        #print ("Topics: " + topics)
        #Create 3 Category columns
        all_topics = topics.split(", ")
        if ('Climate Change' in all_topics):
            all_topics.remove('Climate Change')
        if ('Renewable Energy' in all_topics):
            all_topics.remove('Renewable Energy')
        if ('Fossil Energy' in all_topics):
            all_topics.remove('Fossil Energy')
        if ('Energy Efficiency' in all_topics):
            all_topics.remove('Energy Efficiency')
        catg1 = ""
        catg2 = ""
        catg3 = ""
        if (len(all_topics) == 1):
            catg1 = all_topics[0]
        if (len(all_topics) == 2):
            catg1 = all_topics[0]
            catg2 = all_topics[1]
        if (len(all_topics) == 3):
            catg1 = all_topics[0]
            catg2 = all_topics[1]
            catg3 = all_topics[2]
        #Associated Bills there sometimes, check this condition and create a column for Associated Bills
        associated_bills = ""
        final_bills = []
        if ("<b>Associated" in lion[7]):
            for bill in lion[7]:
                clean_bills = bill.replace("</b>","")
                final_bills.append(clean_bills)
            for i in range (2, len(final_bills)):
                associated_bills += final_bills[i] + " "
            #print ("Associated Bills: " + associated_bills)
        #Create column Summary;
        #the summary list is at either lion[7]or[8] depending on Associated's apperance
        summary = ""
        if ("<b>Summary:" in lion[7]):
            for i in range (2, len(lion[7])):
                summary += lion[7][i] + " "
            #print ("Summary: " + summary)
        else: 
            for i in range (2, len(lion[8])):
                summary += lion[8][i] + " "
            #print ("Summary: " + summary)            
        #Create history field:
        #look for the first list of the History lists; this list is a bit different than the others
        ###the first history list is at either lion[9]or[10] depending on Associated's appearance
        if ("<b>History:" in lion[8]):  
            history_list = lion[9]
            #print ("History: ")
            first_history=""
            for i in range (6,(len(history_list))):
                first_history += history_list[i] + " "
            #print (first_history)

            group_history=[]
            complete_history = lion[10:]
            if len(complete_history)>1:
                for history in complete_history:
                    all_history=""
                    if history == complete_history[-2]:
                        for i in range (0,len(history)-1):
                            all_history += history[i] + " "
                        #print (all_history)
                        group_history.append(all_history)
                    else:        
                        for i in range (0, len(history)):
                            all_history += history[i] +" "
                        group_history.append(all_history)
                        #print (all_history)
        else:
            history_list = lion[10]
            #print ("History: ")
            first_history=""
            for i in range (6,(len(history_list))):
                first_history += history_list[i] + " "
            #print (first_history)

            group_history=[]
            complete_history = lion[11:]
            if len(complete_history)>1:
                for history in complete_history:
                    all_history=""
                    if history == complete_history[-2]:
                        for i in range (0,len(history)-1):
                            all_history += history[i] + " "
                        group_history.append(all_history)
                        #print (all_history)
                    else:        
                        for i in range (0, len(history)):
                            all_history += history[i] +" "
                        group_history.append(all_history)
                        #print (all_history)
        history_field = ""
        for i in range (0,len(group_history)):
            history_field += group_history[i]
        all_hist_field = (first_history + history_field)
        #print ("History: " + all_hist_field)
        
        updated = 'none'
        
        #date_added = dt.datetime.today().strftime("%m/%d/%Y")
        date_added = '09/09/15'
        
        newrow = [bill_name, bill_state, bill_chamber, bill_number, bill_one
                  , bill_year, title, 
                  status, status_one, status_two, status_three
                  , date, authors_name, aff_authors, 
                  coauthors1_name, aff_coauthors1, 
                  coauthors2_name, aff_coauthors2,
                  coauthors3_name, aff_coauthors3,
                  other_coauthors,
                  topics, catg1, catg2, catg3
                  , associated_bills, summary, all_hist_field, updated, date_added]
        bill_rows.append(newrow)
        #print()
        #print()
    a.writerows(bill_rows)